In [1]:
using DelimitedFiles

In [2]:
kernel = convert.(Bool, [
     0 1 0;
     1 0 1;
     0 1 0]
    )

function kernel_indices(point, data; kernel=kernel)
    i, j = point
    kernel_extent = size(kernel) .÷ 2
    inds = CartesianIndices((i-kernel_extent[1]:i+kernel_extent[1],
                            j-kernel_extent[2]:j+kernel_extent[2]))[kernel]
    inds = filter(ind->checkbounds(Bool, data, ind), inds)
    return inds
end

function solve_part1(filename)
    in_data = readdlm(filename, '\n', String)
    data = parse.(Int, permutedims(hcat(collect.(in_data)...)))
    risk_level = 0
    for i = 1:size(data)[1]
        for j = 1:size(data)[2]
            inds = kernel_indices((i, j), data)
            if data[i, j] < minimum(data[inds])
                risk_level += (data[i, j] + 1)
            end
        end
    end
    return risk_level
end

println("Part 1 test: $(solve_part1("test.txt"))")
println("Part 1 solution: $(solve_part1("input.txt"))")


Part 1 test: 15
Part 1 solution: 508


In [3]:
function basin_size(point, data, val)
    neighbours = kernel_indices(point, data)
    bs = 1
    data[point...] = -1
    for n = neighbours
        if (data[n] != 9) & (data[n] != -1)
            bs += basin_size(Tuple(n), data, val)
        end
    end
    return bs
end

function solve_part2(filename)
    in_data = readdlm(filename, '\n', String)
    data = parse.(Int, permutedims(hcat(collect.(in_data)...)))
    basin_sizes = []
    for i = 1:size(data)[1]
        for j = 1:size(data)[2]
            inds = kernel_indices((i, j), data)
            if data[i, j] < minimum(data[inds])
                bs = basin_size((i, j), data, length(basin_sizes)+1)
                append!(basin_sizes, bs)
            end
        end
    end
    return prod(sort(basin_sizes)[end-2:end])
end

println("Part 2 test: $(solve_part2("test.txt"))")
println("Part 2 solution: $(solve_part2("input.txt"))")

Part 2 test: 1134
Part 2 solution: 1564640
